##  ***Ejemplo Calibración y Pricing Modelo HW***



In [69]:
from curve_builder import Curve
from hw_model import HullWhiteModel, HullWhiteCurveBuilder, HullWhitePricer, HullWhiteSimulation
from calibration import HullWhiteCalibrator

import numpy as np
import pandas as pd 
import plotly.express as px 
import QuantLib as ql 

***1. Lectura de inputs (EUR)***

In [70]:
# Curvas de mercado 
input_curve = pd.read_excel(r'Plantilla Calibracion EUR.xlsx')
time = input_curve['Year Frac.']
disc_rate = input_curve['Discount Factor']

# Leemos la plantilla para la calibracion
df = pd.read_excel(r'Plantilla Calibracion EUR.xlsx', sheet_name = 'Plantilla')
date_columns = df.columns[4:]

market_prices = {
    'Price': df['Price'].tolist(),
    'Strike': df['Strike'].tolist(),
    'Notional': df['Notional'].tolist(),
    'Dates': df[date_columns].apply(lambda row: row.dropna().tolist(), axis=1).tolist()}

***2. Construcción e interpolación de curvas***

In [ ]:
# Prueba de la clase Curve
curve = Curve(time, disc_rate)
t = np.linspace(0, 30, 500)
px.line(x = t, y = curve.forward(t), title = "Curva Forward Instantánea (EUR)") 

***3. Calibracion a mercado (EUR)***

In [72]:
init_params = {'a': 0.01, 'sigma':0.01, 'r0': curve.forward(0)}

# Inicializamos modelo, simulador, y creador de curvas
hw_model = HullWhiteModel(curve, init_params)
hw_sim = HullWhiteSimulation(hw_model)
hw_curve = HullWhiteCurveBuilder(hw_model, hw_sim)

# Inicializamos el pricer, el calibrador y calibramos a mercado 
pricer = HullWhitePricer(hw_model, hw_curve)
calibrator = HullWhiteCalibrator(hw_model, pricer, market_prices)
result = calibrator.calibrate() 

a: 0.06449, sigma: 0.00950, Error: 3.62103e-02
a: 0.04420, sigma: 0.00917, Error: 2.08750e-02
a: 0.01272, sigma: 0.00860, Error: 9.27938e-03
a: 0.01269, sigma: 0.00859, Error: 9.27837e-03
a: 0.01266, sigma: 0.00859, Error: 9.27814e-03
 
Diferencias Cap 2y:  6.6844%
Diferencias Cap 3y: -2.8650%
Diferencias Cap 4y: -4.9987%
Diferencias Cap 10y: -3.0500%
Diferencias Cap 20y:  2.3667%


***4. Simulación de escenarios consistentes con mercado***

In [ ]:
# Actualizamos el modelo con los nuevos parametros 
a_opt, sigma_opt = result.x
params = {'a': a_opt, 'sigma':sigma_opt, 'r0': curve.forward(0)}
hw_model = HullWhiteModel(curve, params)
hw_sim = HullWhiteSimulation(hw_model, n_paths = 10**6) 

# Simulacion de la short rate r(t)
r_t = hw_sim.simulate_short_rate_direct(T = 1) 
fig = px.histogram(r_t, nbins = 200, title = 'Distribución de la short rate en un año')
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
# Simulacion del tipo a plazo R(t, T)
hw_curve = HullWhiteCurveBuilder(hw_model, hw_sim)
R_t = hw_curve.long_rate(t = 1, T = 20) 
fig = px.histogram(R_t, nbins = 200, title = 'Distribución del tipo a 20 años en un año')
fig.update_layout(showlegend=False)
fig.show()

***5. Valoración de instrumentos Vanilla***

In [75]:
pricer = HullWhitePricer(hw_model, hw_curve)

# Opcion call sobre un bono
bond_call = pricer.zero_bond_call(T = 1, S = 2, K = 0.9)

# Cap anual
Tau = [0, 1, 2, 3, 4, 5]; N = 1; K = 0.025
cap = pricer.cap(Tau, N, K)

***6. Valoración de instrumentos OTC***

*EUR 100,000,000 2.04700000% Cap on 3-Month EUR-EURIBOR*

*EUR 100,000,000 2.04700000% Floor on 3-Month EUR-EURIBOR*

*Effective 20-Sep-2026 through 20-Sep-2027*

*MTM:  831,056.60 €*


In [76]:
# Fechas de inicio y fin
val_date = ql.Date(31, 3, 2025)
start_date = ql.Date(20, 9, 2026)
end_date   = ql.Date(20, 9, 2027)
day_count = ql.Actual360()

# Fechas de pago (la primera no paga)
schedule = ql.Schedule(val_date, end_date, ql.Period(ql.Quarterly), ql.TARGET(), ql.Following, ql.Following, ql.DateGeneration.Forward, False)
Tau = [day_count.yearFraction(val_date, d) for d in schedule if d >= start_date]

# Strike y nominal
K = 0.02047; N = 100_000_000

# Valoracion
mtm = 831_056.60
cap = pricer.cap(Tau, N, K)
floor = pricer.floor(Tau, N, K)
straddle = cap + floor
dif = straddle/mtm - 1
print(f"Error cometido :{100*dif: .4f}%")

Error cometido : 6.2403%


El error de la valoración va en linea con el error en los instrumentos de la calibración. El modelo es **monocurva**, por tanto, genera los fwd a partir de la **OIS ESTR** y no del **EUR EURIBOR 3M**, lo que puede estar aumentando el error.

***7. Calibración a USD***



In [77]:
# Curvas de mercado 
input_curve = pd.read_excel(r'Plantilla Calibracion USD.xlsx').dropna()
time = input_curve['Year Frac.']
disc_rate = input_curve['Discount Factor']

# Leemos la plantilla para la calibracion
df = pd.read_excel(r'Plantilla Calibracion USD.xlsx', sheet_name = 'Plantilla')
date_columns = df.columns[4:]

market_prices = {
    'Price': df['Price'].tolist(),
    'Strike': df['Strike'].tolist(),
    'Notional': df['Notional'].tolist(),
    'Dates': df[date_columns].apply(lambda row: row.dropna().tolist(), axis=1).tolist()}

In [ ]:
# Prueba de la clase Curve
curve = Curve(time, disc_rate)
t = np.linspace(0, 30, 500)
px.line(x = t, y = curve.forward(t), title = "Curva Forward Instantánea (USD)") 

In [79]:
init_params = {'a': 0.01, 'sigma':0.01, 'r0': curve.forward(0)}

# Inicializamos modelo, simulador, y creador de curvas
hw_model = HullWhiteModel(curve, init_params)
hw_sim = HullWhiteSimulation(hw_model)
hw_curve = HullWhiteCurveBuilder(hw_model, hw_sim)

# Inicializamos el pricer, el calibrador y calibramos a mercado 
pricer = HullWhitePricer(hw_model, hw_curve)
calibrator = HullWhiteCalibrator(hw_model, pricer, market_prices)
result = calibrator.calibrate() 

a: 0.00015, sigma: 0.01069, Error: 2.03142e-02
a: 0.00670, sigma: 0.01078, Error: 1.42768e-02
a: 0.03044, sigma: 0.01134, Error: 3.90414e-03
a: 0.03328, sigma: 0.01149, Error: 3.03009e-03
a: 0.03555, sigma: 0.01161, Error: 2.82216e-03
 
Diferencias Cap 2y:  3.7897%
Diferencias Cap 3y: -2.7659%
Diferencias Cap 4y: -2.3994%
Diferencias Cap 10y:  0.1057%
Diferencias Cap 20y:  0.6644%


Mucho menos error en la calibración de USD.

***8. Valoración de instrumentos OTC (USD)***

*Notional Amount: USD 25,000,000.00*

*Effective Date: 03/04/2024*

*Termination Date: 03/04/2029*

*Monthly payments*

*Floor Rate: 3.00%*

*Floating Rate Option: USD-SOFR-OIS Compound 3M*

*MTM: 313,786.47 EUR*

In [80]:
# Fechas de inicio y fin
val_date = ql.Date(31, 3, 2025)
start_date = ql.Date(3, 4, 2024)
end_date = ql.Date(3, 4, 2029)
day_count = ql.Actual360()

# Fechas de pago (la primera no paga)
schedule = ql.Schedule(start_date, end_date, ql.Period(ql.Monthly), ql.TARGET(), ql.Following, ql.Following, ql.DateGeneration.Forward, False)
Tau = [day_count.yearFraction(val_date, d) for d in schedule if d > val_date]

# Strike, nominal, tipo de cambio
K = 0.03; N = 25_000_000; eurusd = 1.0817

# Valoracion
mtm =  313_786.47 
floor = pricer.floor(Tau, N, K)/eurusd
dif = floor/mtm - 1
print(f"Error cometido :{100*dif: .4f}%")

Error cometido :-3.3326%


Error en línea con la calibración. Hay cierto ajuste de convexidad por ser pagos mensuales y la OIS componerse trimestralmente. El primer floorlet no se está calculando (revisar eso en el pricer de floors), debería aumentar el valor y mejorar el error.

***Mejoras***:

- La valoracion por mc de caps y floors hay que revisarla
- Revisar los primeros pagos del pricer analitico de caps/floors
- Implementar funciones de test
- Implementar simulaciones y pricers bajo $Q^T$
- Implementar curvas forward no instantaneas
- Añadir una curva adicional (p.ej EURIBOR)
- Implementar pricers de bonos con cupones, swaps, swaptions
- Mejorar la calibracion (mas instrumentos, mas liquidos, y mejorar optimizacion)
- Escalar a modelo bifactorial (una vez este implementado lo anterior)